In [1]:
import os
import pandas as pd
import numpy as np
import re
import string

In [2]:
path = 'graph_Amazon_Ali/'

In [3]:
################################# Review Node ###################################
reviews = pd.read_csv(path + 'clean/processed_reviews.csv')

reviews

,reviewer_id,product_asin,fake_asin,during_campaign,is_train,removed_by_Amazon,number_of_photos,number_of_helpful,review_date,month_posted,review_rating,has_photos,has_helpful,len_review_title,len_review_body,len_review_text,review_text,product_category_condensed
0,723414,B07K7MSV25,0,0,1,0,0,0,2020-05-07,5,1.0,0.0,0.0,1.0,3.0,4.0,unfriendly behave badly vx:hytfy89usjmys3548,Other
1,1034254,B07H3PDWVZ,0,0,1,1,2,94,2020-06-22,6,5.0,1.0,1.0,3.0,79.0,82.0,waterproof and lightweight. This bottle is 1...,Sports and Outdoors
2,1027740,B07Z3FG46V,1,0,1,1,0,0,2020-05-30,5,5.0,0.0,0.0,4.0,16.0,20.0,Great for multiple devices Awesome to charge a...,Electronics
3,146762,B07RT5DZ2Y,0,0,1,1,0,0,2020-09-23,9,5.0,0.0,0.0,4.0,17.0,21.0,Easy and saves room So simple to use! When d...,Sports and Outdoors
4,627419,B07WHGZLK8,1,1,1,1,0,0,2020-05-21,5,5.0,0.0,0.0,3.0,51.0,54.0,Light house cleaning. Really easy to assemble....,Personal Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398069,921326,B07X8V3SLM,0,0,0,0,0,0,2020-01-12,1,5.0,0.0,0.0,6.0,93.0,99.0,Very Happy with this USB-C Hub I am using th...,Electronics
1398070,572598,B082DBC2X1,1,0,0,0,0,0,2019-12-30,12,5.0,0.0,0.0,7.0,153.0,160.0,"Brilliant design, quality build, looks great t...",Home Improvement
1398071,210516,B084ZDPNTT,1,0,1,0,0,0,2020-09-04,9,5.0,0.0,0.0,2.0,15.0,17.0,Real nice Great quality and feels really nic...,Electronics
1398072,500627,B071PDY5DS,0,0,1,0,0,0,2019-07-29,7,5.0,0.0,0.0,12.0,41.0,53.0,There is a slight taste to them but you barely...,Personal Care


In [16]:
review_features = reviews.drop(columns=['reviewer_id', 'product_asin', 'fake_asin', 'during_campaign', 'is_train', 'removed_by_Amazon', 'review_date', 'has_photos', 'has_helpful', 'review_text'])
#review_features = pd.concat([review_features.drop(columns=['month_posted']), pd.get_dummies(review_features['month_posted']).astype(float)], axis=1)
review_features.drop(columns=['month_posted'], inplace=True)
#review_features = pd.concat([review_features.drop(columns=['product_category_condensed']), pd.get_dummies(review_features['product_category_condensed']).astype(float)], axis=1)
review_features.drop(columns=['product_category_condensed'], inplace=True)
                     
review_features

,number_of_photos,number_of_helpful,review_rating,len_review_title,len_review_body,len_review_text
0,0,0,1.0,1.0,3.0,4.0
1,2,94,5.0,3.0,79.0,82.0
2,0,0,5.0,4.0,16.0,20.0
3,0,0,5.0,4.0,17.0,21.0
4,0,0,5.0,3.0,51.0,54.0
...,...,...,...,...,...,...
1398069,0,0,5.0,6.0,93.0,99.0
1398070,0,0,5.0,7.0,153.0,160.0
1398071,0,0,5.0,2.0,15.0,17.0
1398072,0,0,5.0,12.0,41.0,53.0


In [38]:
np.save(path + 'nodes-review.npy', review_features.to_numpy())

In [5]:
################################# Product Node ###################################
products_df = pd.read_csv(path + 'clean/processed_products.csv')
products_df

,product_asin,n_of_reviews,share_photos,share_helpful,share_1star,share_5star,avg_review_rating,stdev_review_rating,min_review_rating,max_review_rating,...,3,4,5,6,7,8,9,10,11,12
0,B07K7MSV25,1265,0.126482,0.104348,0.101976,0.662451,4.192885,1.343331,1.0,5.0,...,0.050593,0.077470,0.120158,0.111462,0.132016,0.110672,0.081423,0.069565,0.040316,0.065613
1,B07H3PDWVZ,286,0.080420,0.199301,0.139860,0.583916,3.961538,1.485104,1.0,5.0,...,0.076923,0.080420,0.069930,0.104895,0.104895,0.059441,0.083916,0.118881,0.083916,0.083916
2,B07Z3FG46V,1519,0.201448,0.094799,0.150099,0.698486,4.105991,1.512342,1.0,5.0,...,0.144174,0.103357,0.174457,0.090849,0.079658,0.046083,0.032258,0.088874,0.023700,0.042791
3,B07RT5DZ2Y,131,0.038168,0.091603,0.091603,0.564885,4.076336,1.316279,1.0,5.0,...,0.068702,0.022901,0.053435,0.007634,0.030534,0.053435,0.061069,0.083969,0.106870,0.183206
4,B07WHGZLK8,671,0.187779,0.156483,0.101341,0.766021,4.323398,1.348754,1.0,5.0,...,0.053651,0.068554,0.119225,0.099851,0.129657,0.084948,0.108793,0.144560,0.032787,0.058122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403,B0881NL6HH,1,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000,5.0,5.0,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3404,B082F65KF8,2,0.000000,0.500000,0.500000,0.000000,2.500000,2.121320,1.0,4.0,...,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3405,B0822QT8PG,2,0.500000,0.500000,0.500000,0.500000,3.000000,2.828427,1.0,5.0,...,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
3406,B07L5P4HPR,1,0.000000,0.000000,0.000000,1.000000,5.000000,0.000000,5.0,5.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
products_df.iloc[:, 1:18].columns

Index(['n_of_reviews', 'share_photos', 'share_helpful', 'share_1star',
       'share_5star', 'avg_review_rating', 'stdev_review_rating',
       'min_review_rating', 'max_review_rating', 'avg_review_length',
       'stdev_review_length', 'min_review_length', 'max_review_length',
       'avg_days_between_reviews', 'stdev_days_between_reviews',
       'min_days_between_reviews', 'max_days_between_reviews'],
      dtype='object')

In [8]:
np.save(path + 'nodes-product.npy', products_df.iloc[:, 1:18].to_numpy())

In [17]:
################################# Reviewer Node ###################################
reviewers_df = pd.read_csv(path + 'clean/processed_reviewers.csv')
reviewers_df.iloc[:, 0:17]

,n_of_reviews,share_photos,share_helpful,share_1star,share_5star,avg_review_rating,stdev_review_rating,min_review_rating,max_review_rating,avg_review_length,stdev_review_length,min_review_length,max_review_length,avg_days_between_reviews,stdev_days_between_reviews,min_days_between_reviews,max_days_between_reviews
0,1.0,0.0,0.0,0.0,1.0,5.0,0.0,5.0,5.0,28.0,0.000000,28.0,28.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,5.0,0.0,5.0,5.0,18.0,0.000000,18.0,18.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,5.0,0.0,5.0,5.0,58.0,0.000000,58.0,58.0,0.0,0.0,0.0,0.0
3,2.0,0.0,0.5,0.0,1.0,5.0,0.0,5.0,5.0,47.5,0.707107,47.0,48.0,39.0,0.0,39.0,39.0
4,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,11.0,0.000000,11.0,11.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281354,1.0,0.0,0.0,0.0,1.0,5.0,0.0,5.0,5.0,21.0,0.000000,21.0,21.0,0.0,0.0,0.0,0.0
1281355,1.0,0.0,0.0,0.0,1.0,5.0,0.0,5.0,5.0,18.0,0.000000,18.0,18.0,0.0,0.0,0.0,0.0
1281356,1.0,0.0,1.0,0.0,1.0,5.0,0.0,5.0,5.0,33.0,0.000000,33.0,33.0,0.0,0.0,0.0,0.0
1281357,1.0,1.0,0.0,0.0,1.0,5.0,0.0,5.0,5.0,28.0,0.000000,28.0,28.0,0.0,0.0,0.0,0.0


In [9]:
np.save(path + 'nodes-reviewer.npy', reviewers_df.iloc[:, 0:17].to_numpy())

In [6]:
################################# Edges ###################################
map_ = reviews[['reviewer_id', 'product_asin']]

product_dict = products_df[['product_asin']]
product_dict['product_id'] = products_df.index
product_dict

/tmp/ipykernel_71026/3322115303.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_dict['product_id'] = products_df.index


,product_asin,product_id
0,B07K7MSV25,0
1,B07H3PDWVZ,1
2,B07Z3FG46V,2
3,B07RT5DZ2Y,3
4,B07WHGZLK8,4
...,...,...
3403,B0881NL6HH,3403
3404,B082F65KF8,3404
3405,B0822QT8PG,3405
3406,B07L5P4HPR,3406


In [7]:
product_dict.to_csv('product_ids.csv')

In [8]:
edge_map = map_.merge(product_dict, left_on='product_asin', right_on='product_asin', how='left').drop(columns='product_asin')
edge_map['review_id'] = edge_map.index
edge_map

,reviewer_id,product_id,review_id
0,723414,0,0
1,1034254,1,1
2,1027740,2,2
3,146762,3,3
4,627419,4,4
...,...,...,...
1398069,921326,541,1398069
1398070,572598,386,1398070
1398071,210516,115,1398071
1398072,500627,2143,1398072


In [9]:
edge_map.to_csv('edge_map.csv')

In [18]:
#edge_map[['reviewer_id', 'review_id']].drop_duplicates().to_csv(path + 'edges-to-review-from-reviewer.csv', index=False, header=['Source', 'Target'])
#edge_map[['review_id', 'product_id']].drop_duplicates().to_csv(path + 'edges-to-product-from-review.csv', index=False, header=['Source', 'Target'])
edge_map[['reviewer_id', 'product_id']].drop_duplicates().to_csv(path + 'edges-to-product-from-reviewer.csv', index=False, header=['Source', 'Target'])

# Reverse edges
#edge_map[['review_id', 'reviewer_id']].drop_duplicates().to_csv(path + 'edges-to-reviewer-from-review.csv', index=False, header=['Source', 'Target'])
#edge_map[['product_id', 'review_id']].drop_duplicates().to_csv(path + 'edges-to-review-from-product.csv', index=False, header=['Source', 'Target'])
edge_map[['product_id', 'reviewer_id']].drop_duplicates().to_csv(path + 'edges-to-reviewer-from-product.csv', index=False, header=['Source', 'Target'])

# Product-Product edges
product_product = pd.read_csv(path + 'raw/product_edges.csv')
temp = pd.read_csv(path + 'clean/product_map.csv').to_dict()['product_asin']
product_dict = {v: k for k, v in temp.items()}
product_product.applymap(lambda x: product_dict.get(x)).drop_duplicates().to_csv(path + 'edges-to-product-from-product.csv', index=False, header=['Source', 'Target'])

In [24]:
categories_map = reviews[['product_category_condensed']].merge(edge_map, left_index=True, right_on='review_id', how='left')
categories_map

,product_category_condensed,reviewer_id,product_id,review_id
0,Other,723414,0,0
1,Sports and Outdoors,1034254,1,1
2,Electronics,1027740,2,2
3,Sports and Outdoors,146762,3,3
4,Personal Care,627419,4,4
...,...,...,...,...
1398069,Electronics,921326,541,1398069
1398070,Home Improvement,572598,386,1398070
1398071,Electronics,210516,115,1398071
1398072,Personal Care,500627,2143,1398072


In [25]:
for cat in categories_map.product_category_condensed.unique():
    categories_map.loc[categories_map.product_category_condensed == cat, ['review_id', 'product_id']].drop_duplicates().to_csv(path + 'edges-' + cat + '-to-product-from-review.csv', index=False, header=['Source', 'Target'])
    categories_map.loc[categories_map.product_category_condensed == cat, ['product_id', 'review_id']].drop_duplicates().to_csv(path + 'edges-' + cat + '-to-review-from-product.csv', index=False, header=['Source', 'Target'])
    


In [12]:
edge_map[['reviewer_id', 'product_id']].drop_duplicates().to_csv(path + 'edges-to-product-from-reviewer.csv', index=False, header=['Source', 'Target'])